In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [2]:
raw_data_dir = Path.cwd() / "data" / "raw"
clean_data_dir = Path.cwd() / "data" / "clean"

# Taylor Rule

In [16]:
time_span = [year for year in range(1980, 2003)]

### Interest Rates

(OECD)

In [41]:
df = pd.read_csv(
    raw_data_dir / "ST_interestrate_1980-2002.csv",
    sep=";",  
)

In [42]:
df = df.rename(columns={"Year": "year", "Interest Rate": "interest_rate"})

In [43]:
df = df.loc[df["year"].isin(time_span)]

In [47]:
df["interest_rate"] = df["interest_rate"].apply(lambda x: x.replace(",", ".")).astype(float)

In [50]:
df.to_csv(clean_data_dir / "interest_rate.csv", sep="\t", index=False)

### Inflation

(WWW.RIVALUTA.it)

In [9]:
df = pd.read_csv(
    raw_data_dir / "datiinflazionemediaitalia.csv", 
    sep=";",  
)

In [10]:
df = df[1:-1]

In [11]:
df = df.rename(columns={"WWW.RIVALUTA.it": "inflation_rate"})

In [12]:
df.reset_index(col_level=0, names="year", inplace=True)

In [13]:
df = df.sort_values("year")

In [14]:
df["year"] = df["year"].astype(int)

In [15]:
df = df.loc[df["year"].isin(time_span)]

In [24]:
df["inflation_rate"] = df["inflation_rate"].apply(
    lambda x: float(x.replace(",", ".").strip("%")) / 100.0
)

AttributeError: 'float' object has no attribute 'replace'

In [25]:
df.head()

,year,inflation_rate
43,1980,0.212
42,1981,0.178
41,1982,0.165
40,1983,0.147
39,1984,0.108


In [26]:
df.to_csv(clean_data_dir / "inflation_rate.csv", sep="\t", index=False)

### Gdp

(Macrotrends)

In [48]:
df = pd.read_csv(
    raw_data_dir / "italy-gdp-gross-domestic-product.csv",
    sep=",",
    usecols=["date", "GDP ( Billions of US $)"],
)

In [49]:
df = df.rename(columns={"GDP ( Billions of US $)": "gdp"})

In [8]:
def date_to_year(date):
    date_object = datetime.strptime(date, "%Y-%m-%d")
    year = date_object.year
    return year

In [51]:
df["year"] = df["date"].apply(lambda x: date_to_year(x))
df.drop("date", axis=1, inplace=True)

In [52]:
df = df[["year", "gdp"]]

In [54]:
df = df.loc[df["year"].isin(time_span)]

In [55]:
df.head()

,year,gdp
20,1980,477.256776
21,1981,430.702851
22,1982,427.272646
23,1983,443.042374
24,1984,437.887689


In [56]:
df.to_csv(clean_data_dir / "gdp.csv", sep="\t", index=False)

### Output Gap 

(Nasdaq)

In [57]:
df = pd.read_csv(
    raw_data_dir / "nasdaq_output_gap.csv",
    sep=",",
)

In [58]:
df = df.rename(columns={"Value": "output_gap"})

In [59]:
df["year"] = df["Date"].apply(lambda x: date_to_year(x))
df.drop("Date", axis=1, inplace=True)

In [60]:
df = df[["year", "output_gap"]]

In [61]:
df = df.sort_values("year")

In [64]:
df = df.loc[df["year"].isin(time_span)]

In [65]:
df.head()

,year,output_gap
44,1980,12.396
43,1981,-0.922
42,1982,-2.350
41,1983,-3.552
40,1984,-2.860


In [66]:
df.to_csv(clean_data_dir / "output_gap.csv", sep="\t", index=False)

# Additional Regressors

### Unemployment

(FRED)

In [67]:
df = pd.read_csv(
    raw_data_dir / "LRHUTTTTITA156S.csv",
    sep=",",
)

In [68]:
df = df.rename(columns={"DATE": "year", "LRHUTTTTITA156S": "unemployment_rate"})

In [70]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [73]:
df = df.loc[df["year"].isin(time_span)]

In [74]:
df.head()

,year,unemployment_rate
0,1982,7.166667
1,1983,7.366667
2,1984,7.833333
3,1985,8.141667
4,1986,8.883333


In [75]:
df.to_csv(clean_data_dir / "unemployment_rate.csv", sep="\t", index=False)

### Exchange Rate

In [56]:
df = pd.read_csv(
    raw_data_dir / "USD_serie_storica_giornaliera .csv",
    sep=",",
    usecols=["Data di Riferimento", "Quotazione"]
)

In [57]:
df = df.rename(columns={"Data di Riferimento": "year", "Quotazione": "exchange_rate"})

In [58]:
df = df.sort_values("year")

In [59]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [60]:
df = df.drop_duplicates("year", keep='last')

In [61]:
df = df.loc[df["year"].isin(time_span)]

In [62]:
add = pd.DataFrame(
[[2002, 1950.000]],
    columns=["year", "exchange_rate"]
)
add

,year,exchange_rate
0,2002,1950.0


In [64]:
df = pd.concat([df, add])

In [66]:
df = df.reset_index(drop=True)

In [67]:
df.to_csv(clean_data_dir / "exchange_rate.csv", sep="\t", index=False)

### Foreign Interest Rate

In [31]:
df = pd.read_csv(
    raw_data_dir / "US10y_bond.csv",
    sep=",",
)

In [32]:
df = df.rename(columns={"YEAR": "year", "Value": "US_interest_rate"})

In [33]:
df = df.loc[df["year"].isin(time_span)]

In [34]:
df.head()

,year,US_interest_rate
20,1980,11.460000
21,1981,13.910833
22,1982,13.001667
23,1983,11.105000
24,1984,12.438333


In [35]:
df.to_csv(clean_data_dir / "us_bond_yield.csv", sep="\t", index=False)